In [1]:

import numpy as np
import sys
import imutils
import cv2, PIL
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import cv2.aruco as aruco
import time 



In [96]:
def imgDetection():
    total = 0
    #cap = cv2.VideoCapture(0)

    frame = cv2.imread('test.jpg')
    frame = cv2.resize(frame, None, fx=0.2, fy=0.2, interpolation=cv2.INTER_CUBIC) # resize img
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
    parameters = aruco.DetectorParameters_create()
    cYArray = []


    #print(parameters)

    '''    detectMarkers(...)
        detectMarkers(image, dictionary[, corners[, ids[, parameters[, rejectedI
        mgPoints]]]]) -> corners, ids, rejectedImgPoints
        '''
        #lists of ids and the corners beloning to each id
    #corners, idss, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    gray = aruco.drawDetectedMarkers(gray, corners)
    frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, ids)
    # verify *at least* one ArUco marker was detected
    if len(corners) > 0 and len(frame) > 0:
        total = total + 1
        #print(ids , "\n")
        # flatten the ArUco IDs list
        ids = ids.flatten()

        sort_ids=[]
        for(markerCorner, markerID) in zip(corners, ids):

            # extract the marker corners (which are always returned in
            # top-left, top-right, bottom-right, and bottom-left order)
            corners = markerCorner.reshape((4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = corners
            # convert each of the (x, y)-coordinate pairs to integers
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))
            # compute and draw the center (x, y)-coordinates of the
            # ArUco marker
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
            
            sort_ids.append((markerID,cX,cY))
        
        #Sort left to right and top to bottom
        #(id,x,y)
        print(sort_ids)    
        print(sorted(sort_ids, key = lambda x: x[1] + 3 * x[2])) 

    #print(rejectedImgPoints)
    # Display the resulting frame
    cv2.imshow("ArUco Board",frame)


    # When everything done, release the capture

    #cap.release()
    cv2.destroyAllWindows()

In [97]:
imgDetection()

ids:
[(6, 482, 619), (14, 264, 606), (13, 270, 463), (15, 278, 325), (7, 282, 183), (3, 480, 478), (2, 489, 331)]
sort:
[(7, 282, 183), (15, 278, 325), (2, 489, 331), (13, 270, 463), (3, 480, 478), (14, 264, 606), (6, 482, 619)]


In [ ]:
def motionDetection():
    # Seems to be better, but I need to write an algorithm to process too much data
    
    total = 0
    cap = cv2.VideoCapture(0)

    while(total < 1000):
        # Capture frame-by-frame
        ret, frame = cap.read()


        #print(frame.shape) #480x640
        # Our operations on the frame come here
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
        parameters =  aruco.DetectorParameters_create()

        #print(parameters)

        '''    detectMarkers(...)
            detectMarkers(image, dictionary[, corners[, ids[, parameters[, rejectedI
            mgPoints]]]]) -> corners, ids, rejectedImgPoints
            '''
            #lists of ids and the corners beloning to each id
        corners, idss, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        gray = aruco.drawDetectedMarkers(gray, corners)
        frame_markers = aruco.drawDetectedMarkers(frame.copy(), corners, idss)
        # verify *at least* one ArUco marker was detected
        if len(corners) > 0 and len(frame) > 0:
            total = total + 1
            print(idss , "\n")
            # flatten the ArUco IDs list
            ids = idss.flatten()


            for(markerCorner, markerID) in zip(corners, ids):

                # extract the marker corners (which are always returned in
                # top-left, top-right, bottom-right, and bottom-left order)
                corners = markerCorner.reshape((4, 2))
                (topLeft, topRight, bottomRight, bottomLeft) = corners
                # convert each of the (x, y)-coordinate pairs to integers
                topRight = (int(topRight[0]), int(topRight[1]))
                bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
                bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
                topLeft = (int(topLeft[0]), int(topLeft[1]))

                # draw the bounding box of the ArUCo detection
                cv2.line(frame, topLeft, topRight, (0, 255, 0), 2)
                cv2.line(frame, topRight, bottomRight, (0, 255, 0), 2)
                cv2.line(frame, bottomRight, bottomLeft, (0, 255, 0), 2)
                cv2.line(frame, bottomLeft, topLeft, (0, 255, 0), 2)
                # compute and draw the center (x, y)-coordinates of the
                # ArUco marker
                cX = int((topLeft[0] + bottomRight[0]) / 2.0)
                cY = int((topLeft[1] + bottomRight[1]) / 2.0)
                cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
                # draw the ArUco marker ID on the frame
                print(markerID , "\n")
                cv2.putText(frame, str(markerID),
                            (topLeft[0], topLeft[1] - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255, 0, 0), 2)
        '''size_of_marker =  0.0145 # side lenght of the marker in meter
        rvecs,tvecs, trash = aruco.estimatePoseSingleMarkers(corners, size_of_marker , mtx, dist)'''     




        #print(rejectedImgPoints)
        # Display the resulting frame

        cv2.imshow("ArUco Board",frame)



        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
def detectionSequence():
    